## 크롤링 1
- 유튜브를 크롤링하여 데이터를 획득
- 크롤링을 해야하는 이유는 자체가 ajax를 이용한 방식이므로 사람의 손길이 필요한 사이트
- 일반적인 html 자체에는 자체 정의 태그들을 혼용한다 => Angular js를 부분 사용을 했다라고 판단됨
- 일반적인 css selector의 규칙중에 id규칙(문서내에 고유하다)을 무시한다.(id사용은 자제해야한다)
  => 사용주의(id가지고 한놈만 특정할 수 없다)
- 데이터 로드는 ajax를 통해서 사용자가 스크롤링을 해서 리스트의 끝에 도착할 때 더보기(이런 기능)이 처리되므로, 전체 데이터를 수집하기 위해서는 js를 가동시켜야 한다
- 사용라이브러리는 selenium을 사용 
- https://www.seleniumhq.org/ -> 다운로드 -> third -> 크롬드라이브 -> 다운로드 -> 76버전 -> win32다운
- 웹드라이버(여기서는 크롬을 사용)
  > https://sites.google.com/a/chromium.org/chromedriver/downloads
- 최신버전이 적용안되는 혹은 버그가 있는 기능이 있으므로, 혹시 안되면 계속 버전을 내려가면서 사용하면된다(안정화가 덜 되서 그럼)
- API-DOCS
  > https://sites.google.com/a/chromium.org/docs/api/py/index.html
  > https://selenium-python.readthedocs.io/ : 지침서
- install
  > conda install selenium
  > or
  > pip install selenium


In [4]:
from selenium import webdriver as wd

In [5]:
import urllib

In [131]:
# 유뷰브 데이터가져올 때 필터 오늘것으로 했을 경우 관점 2개로 나뉨
# 관점 1. 오늘 데이터를 디비에 넣은후에 더보기로 해서 가져오는것
# 관점 2. 더보기없이 전부 디비에 더 집어넣은후 더이상 나오지 않을 때까지 끝까지 가져오는것

In [132]:
# 유튭 ytn실시간 -> 실시간 채팅 -> 프레임소스보기 -> 데이터를 가져와서 진보/보수 어느 정도있는지 분석할 수 있다

In [6]:
# 업로드순 정렬, 오늘(현재기준 24시간이내 게시물)
target_url = 'https://www.youtube.com/results?search_query={}&sp=CAISBAgCEAE%253D'

In [7]:
keyword = '갤럭시노트10'

In [8]:
target_url = target_url.format( urllib.parse.quote(keyword) )
target_url

'https://www.youtube.com/results?search_query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%EB%85%B8%ED%8A%B810&sp=CAISBAgCEAE%253D'

In [9]:
# 웹드라이버 가동
driver = wd.Chrome('../tool/chromedriver75.0.3770.140.exe')

In [10]:
# 사이트 접속
driver.get( target_url )

In [11]:
# 스크롤을 쳐서 데이터를 다 로드 시킨다
# 데이터량이 엄청 많지 않을때 가능
# ajax를 사용할 경우 서버가 반응해서 응답이 오고 화면구성할때까지 시간이 걸린다, 스크롤마다 대기시간이 필요하다
import time # 물리적으로 그냥 마냥 기다려라라는 코드
for n in range(100):
    driver.execute_script( 'window.scrollBy(0,1000)' ) # 윈도우가 축소됫을경우, 확대됫을경우 코드가 달라진다
    # 다음 작업을 코드적으로 강제 대기시킨것
    # 데이터 추출시에는 selenium에서 지원하는 waits기법을 사용하면 좋다
    time.sleep(2) # 유튜브 윈도우에서 스크롤리 내려갔다가 올라가는것을 볼 수 있다(결과가 없을때가지 내려감) -> 하지만 아직 코드는 진행

In [139]:
# 영상 데이터 한줄 :
# ytd-video-renderer
# 영상의 메타 정보 :
# #video-title

In [13]:
videos = driver.find_elements_by_tag_name( 'ytd-video-renderer' )
len( videos ), type( videos ) # 튜플로 된다

(79, list)

In [14]:
# driver.find_element_by_id
# driver.find_element_by_css_selector
for video in videos[:2]:
    # 영상의 타이틀
    print( video.find_element_by_id('video-title').get_attribute('title') )
    # 썸네일
    print( video.find_element_by_tag_name('img').get_attribute('src') )
    # 링크
    print( video.find_element_by_id('video-title').get_attribute('href') )
    # 작성자
    print( video.find_element_by_css_selector('#byline>a').text )
    # 조회수 : metadata-line
    for span in video.find_elements_css_selector('#metadata-line>span'):
        print( span.text )
    '''
    [
      {'title':'xx', 'thum':'xx', 'link':'xx', 'author':'xx', 'views':'xx', 'time':''},
      ,{},{}, ...
    ]
    '''
    # 작성시간
    

[개봉기] '갤럭시 노트10+ 256GB 화이트' 자급제 구매했다!
https://i.ytimg.com/vi/8UzFc0WRiZs/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAfMvJgQoASb_PETKx1DtDAP3LJdw
https://www.youtube.com/watch?v=8UzFc0WRiZs
리트 REAT


AttributeError: 'WebElement' object has no attribute 'find_elements_css_selector'

In [ ]:
results = []
for video in videos: #[:2]:
    dic = {
        'title':video.find_element_by_id('video-title').get_attribute('title')
        ,'thum':video.find_element_by_tag_name('img').get_attribute('src') 
        ,'link':video.find_element_by_id('video-title').get_attribute('href')     
        ,'author':video.find_element_by_css_selector('#byline>a').text 
        ,'views':video.find_elements_by_css_selector('#metadata-line>span')[0].text # views는 배열로 온다
        ,'time':video.find_elements_by_css_selector('#metadata-line>span')[1].text
    }    
    results.append( dic )
results

In [ ]:
for link in results[-1:]:
    print( link['link'] ) # 데이터 뽑기 : 인덱싱
    # 해당 콘텐츠에 진입
    driver.get( link['link'])
    # 잠시대기 -> 반드시 신규페이지에서는 대기
    print('5초 대기')
    driver.implicitly_wait(5)
    # 화면스크롤 
    for n in range(3):
        print('화면스크롤 시도 ')
        driver.execute_script( 'window.scrollBy(0,1000)' )
    # 후기 개수를 확인 -> 적정선까지 로드
    for cnt in driver.find_elements_by_id('count'):
        print( cnt.text )
    # 후기 채집( 작성자, 내용, 좋아요 )
    for comment in driver.find_elements_by_tag_name('ytd-comment-thread-renderer'):
        print( comment.text )
        print( '-'*20 )

In [ ]:
# 1. 한글명은 동일하게 세팅되니까 replace로 걷어내는 방법
# 2. 정규식을 통해서 숫자만 남기고 모두 제거
tmp = ['조회수 39,529', '댓글 362개']
tmp[0].replace('조회수','').replace('회','')

In [ ]:
tmp[1].replace('댓글','').replace('개','')

In [ ]:
driver.close()  # 항상 꺼줘야 한다
driver.quit()